In [7]:
import numpy as np
import pandas as pd
import copy
import time
from numba import jit
from pwapprx import PWApprx
np.set_printoptions(linewidth=np.inf)

In [8]:
def make_matrix(     ra,
                     rb,
                     n_points = 5, # integer number of (internal) points used to construct a linear apprx, this should be odd
                     savefile = False, # save a csv file of the matrix
                     print_to_log = False
                    ):   
    
    @jit
    def calc_equal_portion(n, total_x_interval):
        p = (1/n) * total_x_interval
        return p

    # returns the weight (the proportion) of x values of a function mapped by y values of another fuction   
    @jit
    def calc_weight(x_lb, x_ub, y_lb, y_ub):
        overlap_lb = max(x_lb,y_lb)
        overlap_ub = min(x_ub,y_ub)

        if overlap_lb > overlap_ub:
            return 0
        else:
            w = (overlap_ub - overlap_lb) /(y_ub - y_lb)
        return w  

    if n_points % 2 == 0:
        n_points = n_points + 1
        print(f"n_points cannot be even, changed n_points to {n_points}")
        
    start = time.time()
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    
    # create functions for linear approximation
        
    func_a = PWApprx(x_values, ra)
    func_b = PWApprx(x_values, rb)

    n_equations = len(x_values)-1
    
    zeros = np.zeros((n_equations,n_equations))
    for z_i, z in enumerate(zeros):
        z[z_i] = -1
    
    p = [(calc_equal_portion(n_equations, total_x_interval)/2) for i in range(n_equations * 2)]
    ans = np.zeros(len(p))
    ans[-1] = 1

    # Calc Lyapunoc using Regular Matrices
    
    y_values_b = func_b.get_y_values()
    top_right_square = []

    for i in range(len(x_values)):
        if i != 0:      
            w_i = []
            for j in range(len(x_values)):
                if j != 0:  
                    # Top Right Square
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_b[j-1],y_values_b[j]),max(y_values_b[j-1],y_values_b[j]))
                    w_i.append(weight)

            top_right_square.append(w_i)

    # needed to compute lyapunov exponent
    slopes_b = func_b.get_slope_list()

    top = np.concatenate((zeros, top_right_square), axis = 1)

    y_values_a = func_a.get_y_values()
    bottom_left_square = []

    for i in range(len(x_values)):
        if i != 0:      
            w_i = []
            for j in range(len(x_values)):
                if j != 0:  
                    # Bottom Left Square
                    weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
                    w_i.append(weight) 
            bottom_left_square.append(w_i)

    bottom = np.concatenate((bottom_left_square, zeros), axis = 1)

    matrix = np.concatenate((top, bottom))
#     matrix[-1] = p

    slopes_a = func_a.get_slope_list()
    slopes = slopes_a + slopes_b
    
    try:
        solution = np.linalg.solve(matrix, ans)
        
        lyapunov_ex = np.sum(p * solution * np.log(np.abs(slopes)))
    except:
        lyapunov_ex = "Singularity"
        solution = "Singularity"
    
    if savefile:
        pd.DataFrame(matrix).to_csv("sample_matrix.csv", header=None, index=None)
    
    if print_to_log:
        print('matrix:')
        print((matrix))
        print(f'ra = {ra} rb = {rb}')
        print(f'slopes: {repr(slopes)}')
        print(f'lyapunov exponent = {lyapunov_ex}')
        print('solution:')
        print(solution)

In [15]:
make_matrix(2.0408163265306123, 2.0, n_points = 101, savefile = True)

matrix:
[[-1.          0.          0.         ...  0.          0.          0.50505051]
 [ 0.         -1.          0.         ...  0.          0.01030928  0.49494949]
 [ 0.          0.         -1.         ...  0.          0.46391753  0.        ]
 ...
 [ 0.          0.          0.         ... -1.          0.          0.        ]
 [ 0.          0.          0.         ...  0.         -1.          0.        ]
 [ 0.          0.          0.         ...  0.          0.         -1.        ]]
ra = 2.0408163265306123 rb = 2.0
slopes: [2.0204081632653064, 1.9795918367346936, 1.940816326530612, 1.9020408163265303, 1.861224489795919, 1.8204081632653075, 1.7795918367346937, 1.7408163265306122, 1.7020408163265286, 1.661224489795919, 1.6204081632653087, 1.579591836734693, 1.5408163265306103, 1.5020408163265317, 1.4612244897959148, 1.4204081632653063, 1.379591836734699, 1.340816326530612, 1.3020408163265318, 1.261224489795919, 1.220408163265304, 1.1795918367346936, 1.1408163265306115, 1.1020408163265292